In [33]:
import pandas as pd
import numpy as np

In [34]:
#%pip install pandas
#%pip install torch
#%pip install sentence-transformers

In [35]:
# below is the datatset annotated by Jay
data = pd.read_csv('final_dataset_06102022 2.csv')

In [36]:
data.head(5)
data.shape

(846, 96)

In [37]:
strs = ["end his life", "end her life", "end my life", "end their lives", "ended his life", "ended her life", "ended my life", "ended their lives", "ending his life", "ending her life", "ending my life", "ending their lives", "ends his life", "ends her life", "ends my life", "ends their lives", "kill oneself", "kill himself", "kill herself", "kill theirselves", "kill myself", "killed oneself", "killed himself", "killed herself", "killed theirselves", "killed myself", "killing oneself", "killing himself", "killing herself", "killing theirselves", "killing myself", "kills oneself", "kills himself", "kills herself", "kills theirselves", "kills myself", "murder oneself", "murder himself", "murder herself", "murder theirselves", "murder myself", "murdered oneself", "murdered himself", "murdered herself", "murdered theirselves", "murdered myself", "murdering oneself", "murdering himself", "murdering herself", "murdering theirselves", "murdering myself", "murders oneself", "murders himself", "murders herself", "murders theirselves", "murders myself", "poison oneself", "poison himself", "poison herself", "poison theirselves", "poison myself", "poisoned oneself", "poisoned himself", "poisoned herself", "poisoned theirselves", "poisoned myself", "poisoning oneself", "poisoning himself", "poisoning herself", "poisoning theirselves", "poisoning myself", "poisons oneself", "poisons himself", "poisons herself", "poisons theirselves", "poisons myself", "drown oneself", "drown himself", "drown herself", "drown theirselves", "drown myself", "drowned oneself", "drowned himself", "drowned herself", "drowned theirselves", "drowned myself", "drowning oneself", "drowning himself", "drowning herself", "drowning theirselves", "drowning myself", "drowns oneself", "drowns himself", "drowns herself", "drowns theirselves", "drowns myself", "strangle oneself", "strangle himself", "strangle herself", "strangle theirselves", "strangle myself", "strangled oneself",  "strangled himself", "strangled herself", "strangled theirselves", "strangled myself", "strangling oneself", "strangling himself", "strangling herself", "strangling theirselves", "strangling myself", "strangles oneself", "strangles himself", "strangles herself", "strangles theirselves", "strangles myself", "suffocate oneself", "suffocate himself", "suffocate herself", "suffocate theirselves", "suffocate myself", "suffocated oneself", "suffocated himself", "suffocated herself", "suffocated theirselves", "suffocated myself", "suffocating oneself", "suffocating himself", "suffocating herself", "suffocating theirselves", "suffocating myself", "suffocates oneself", "suffocates himself", "suffocates herself", "suffocates theirselves", "suffocates myself", "jump to his death", "jump to her death", "jump to their death", "jump to my death", "jumped to his death", "jumped to her death", "jumped to their death", "jumped to my death", "jumping to his death", "jumping to her death", "jumping to their death", "jumping to my death", "jumps to his death", "jumps to her death", "jumps to their death", "jumps to my death", "hang himself", "hang oneself", "hang herself", "hang themselves", "hang myself", "hung himself", "hung oneself", "hung herself", "hung themselves", "hung myself", "hanging himself", "hanging oneself", "hanging herself", "hanging themselves", "hanging myself", "hangs himself", "hangs oneself", "hangs herself", "hangs themselves", "hangs myself", "hungs himself", "hungs oneself", "hungs herself", "hungs themselves", "hungs myself", "shoot oneself", "shoot himself", "shoot herself", "shoot myself", "shoot theirselves", "shot oneself", "shot himself", "shot herself", "shot myself", "shot theirselves", "shooting oneself", "shooting himself", "shooting herself", "shooting myself", "shooting theirselves", "shoots oneself", "shoots himself", "shoots herself", "shoots myself", "shoots theirselves", "shots oneself", "shots himself", "shots herself", "shots myself", "shots theirselves", "die by his hand", "die by his own hand", "die by her hand", "die by her own hand", "die by their hands", "die by their own hands", "die by my hand", "die by my own hand", "take his life", "take his own life", "take her life", "take her own life", "take their lives", "take their own lives", "take my life", "take my own life", "took his life", "took his own life", "took her life", "took her own life", "took their lives", "took their own lives", "took my life", "took my own life", "takes his life", "takes his own life", "takes her life", "takes her own life", "takes their lives", "takes their own lives", "takes my life", "takes my own life", "taking his life", "taking his own life", "taking her life", "taking her own life", "taking their lives", "taking their own lives", "taking my life", "taking my own life","cut oneself", "cut himself", "cut herself", "cut theirselves", "cut myself", "cuts oneself", "cuts himself", "cuts herself", "cuts theirselves", "cuts myself", "cutting oneself", "cutting himself", "cutting herself", "cutting theirselves", "cutting myself", "stab oneself", "stab himself", "stab herself", "stab theirselves", "stab myself", "stabs oneself", "stabs himself", "stabs herself", "stabs theirselves", "stabs myself", "stabbing oneself", "stabbing himself", "stabbing herself", "stabbing theirselves", "stabbing myself", "stabbed oneself", "stabbed himself", "stabbed herself", "stabbed theirselves", "stabbed myself", "cut his throat", "cut her throat", "cut my throat", "cuts his throat", "cuts her throat", "cuts my throat", "cutting his throat", "cutting her throat", "cutting my throat", "cut his wrist", "cut her wrist", "cut my wrist", "cuts his wrist", "cuts her wrist", "cuts my wrist", "cutting his wrist", "cutting her wrist", "cutting my wrist", "slit his throat", "slit her throat", "slit my throat", "slits his throat", "slits her throat", "slits my throat", "slitting his throat", "slitting her throat", "slitting my throat", "slit his wrist", "slit her wrist", "slit my wrist", "slits his wrist", "slits her wrist", "slits my wrist", "slitting his wrist", "slitting her wrist", "slitting my wrist"]

In [38]:
data = data[data['Yes'] == 1]

In [39]:
pattern = r'^news[0-6]_'
datacopy = data.copy()
datacopy['Document name'] = data['Document name'].str.replace(pattern, '', regex=True)

In [40]:
data.head(3)
print(data.shape)

(200, 96)


In [41]:
import torch  
from transformers import BertTokenizer,BertModel

In [42]:
from sentence_transformers import SentenceTransformer,util
from sentence_transformers import models, util, datasets, evaluation, losses
from torch.utils.data import DataLoader

# Define your sentence transformer model using CLS pooling
#model_name = 'sentence-transformers/all-MiniLM-L6-v2'
#word_embedding_model = models.Transformer(model_name)
#print(word_embedding_model.get_word_embedding_dimension())
#pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), 'cls')
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

In [43]:
print(model.max_seq_length)
#model.max_seq_length = 500
print(model.max_seq_length)

384
384


In [44]:
FinancialLabel = "Financial/Job Problem"
Legallabel = "Legal Problem"
SchoolLabel = "School or Academic Related Problem"
HealthLabel = "Lack of Access to Health/Mental Health Care"

In [45]:
relevant = data[data['Yes']==1]
print(relevant.shape)
print(data.shape)

financial_problem = data[data[FinancialLabel] > 0]
school_problem = data[data[SchoolLabel] > 0]
legal_problem = data[data[Legallabel] > 0]
health_problem = data[data[HealthLabel] > 0]
unspecified_circumstance = data[data['Unspecified Circumstance'] > 0]

print("Legal Problem : ", legal_problem.shape[0])
print("Financial/Job Problem : ", financial_problem.shape[0])
print("Lack of Access to Health/Mental Health Care : ", health_problem.shape[0])
print("School or Academic Related Problem : ", school_problem.shape[0])
print("")
print("Other non structural factors: ")

print("Explicit Statement of Mental Health Symptoms or Diagnosis : ", data[data['Explicit Statement of Mental Health Symptoms or Diagnosis'] > 0].shape[0])
print("Social/Relationship Problem : ", data[data['Social/Relationship Problem'] > 0].shape[0])
print("Physical Health Problem : ", data[data['Physical Health Problem'] > 0].shape[0])
print("Crisis plus-minus 2 weeks : ", data[data['Crisis plus-minus 2 weeks'] > 0].shape[0])
print("Loss of Sense of Purpose or Hope: ", data[data['Loss of Sense of Purpose or Hope'] > 0].shape[0])
print("History of Child Abuse : ", data[data['History of Child Abuse'] > 0].shape[0])
print("Lack of Social Support : ", data[data['Lack of Social Support'] > 0].shape[0])
print("Death of Friend or Family Member : ", data[data['Death of Friend or Family Member'] > 0].shape[0])
print("Alcohol Dependence : ", data[data['Alcohol Dependence'] > 0].shape[0])
print("Other Substance Problem : ", data[data['Other Substance Problem'] > 0].shape[0])
print("Disengaged in Activities : ", data[data['Disengaged in Activities'] > 0].shape[0])
print("Access to Means to Harm Self : ", data[data['Access to Means to Harm Self'] > 0].shape[0])
print("Other Circumstance: ", data[data['Other Circumstance'] > 0].shape[0])
print("Unspecified Circumstance: ", data[data['Unspecified Circumstance'] > 0].shape[0])

(200, 96)
(200, 96)
Legal Problem :  39
Financial/Job Problem :  13
Lack of Access to Health/Mental Health Care :  7
School or Academic Related Problem :  5

Other non structural factors: 
Explicit Statement of Mental Health Symptoms or Diagnosis :  35
Social/Relationship Problem :  30
Physical Health Problem :  26
Crisis plus-minus 2 weeks :  8
Loss of Sense of Purpose or Hope:  11
History of Child Abuse :  6
Lack of Social Support :  6
Death of Friend or Family Member :  2
Alcohol Dependence :  2
Other Substance Problem :  7
Disengaged in Activities :  0
Access to Means to Harm Self :  2
Other Circumstance:  8
Unspecified Circumstance:  56


In [46]:
school_problem_df = pd.DataFrame(school_problem, columns=['Document group', 'Document name'])
arrayoftexts = []
wordcount = 0.0
for documentname in school_problem_df['Document name']:
    #print(documentname)
    with open("articles/"+ documentname + ".txt") as f:
        text = f.readlines()
        arrayoftexts.append(text[0])
        wordcount += len(text[0].split())
avgwordcount = wordcount/len(arrayoftexts)

school_problem_df['Text'] = arrayoftexts

school_problem_embeddings = []

for text in school_problem_df['Text']:
    embedding = model.encode(text)
    school_problem_embeddings.append(embedding)

school_problem_averaged_embedding = np.mean(school_problem_embeddings, axis = 0)
print("Average word count: ", avgwordcount)


Average word count:  1285.6


In [47]:
legal_problem_df = pd.DataFrame(legal_problem, columns=['Document group', 'Document name'])
arrayoftexts = []
wordcount = 0.0
for documentname in legal_problem_df['Document name']:
    #print(documentname)
    with open("articles/"+ documentname + ".txt") as f:
        text = f.readlines()
        arrayoftexts.append(text[0])
        wordcount += len(text[0].split())
avgwordcount = wordcount/len(arrayoftexts)

legal_problem_df['Text'] = arrayoftexts

legal_problem_embeddings = []

for text in legal_problem_df['Text']:
    embedding = model.encode(text)
    legal_problem_embeddings.append(embedding)

legal_problem_averaged_embedding = np.mean(legal_problem_embeddings, axis = 0)
print("Average word count: ", avgwordcount)


Average word count:  1058.4615384615386


In [48]:
financial_problem_df = pd.DataFrame(financial_problem, columns=['Document group', 'Document name'])
arrayoftexts = []
wordcount = 0.0
for documentname in financial_problem_df['Document name']:
    #print(documentname)
    with open("articles/"+ documentname + ".txt") as f:
        text = f.readlines()
        arrayoftexts.append(text[0])
        wordcount += len(text[0].split())
avgwordcount = wordcount/len(arrayoftexts)

financial_problem_df['Text'] = arrayoftexts

financial_problem_embeddings = []

for text in financial_problem_df['Text']:
    embedding = model.encode(text)
    financial_problem_embeddings.append(embedding)

financial_problem_averaged_embedding = np.mean(financial_problem_embeddings, axis = 0)
print("Average word count: ", avgwordcount)


Average word count:  1497.6153846153845


In [49]:
health_problem_df = pd.DataFrame(health_problem, columns=['Document group', 'Document name'])
arrayoftexts = []
wordcount = 0.0
for documentname in health_problem_df['Document name']:
    #print(documentname)
    with open("articles/"+ documentname + ".txt") as f:
        text = f.readlines()
        arrayoftexts.append(text[0])
        wordcount += len(text[0].split())
avgwordcount = wordcount/len(arrayoftexts)

health_problem_df['Text'] = arrayoftexts

health_problem_embeddings = []

for text in health_problem_df['Text']:
    embedding = model.encode(text)
    health_problem_embeddings.append(embedding)

health_problem_averaged_embedding = np.mean(health_problem_embeddings, axis = 0)
print("Average word count: ", avgwordcount)


Average word count:  949.5714285714286


In [50]:
import csv
texts = [FinancialLabel, Legallabel, SchoolLabel, HealthLabel] 
embeddingsize = len(financial_problem_averaged_embedding)
header = [i for i in range(embeddingsize)]
header.insert(0, "Structural Violence Factor")



In [51]:
embeddings_final = [financial_problem_averaged_embedding.tolist(), legal_problem_averaged_embedding.tolist(), school_problem_averaged_embedding.tolist(), health_problem_averaged_embedding.tolist()]
for i in range(len(embeddings_final)):
    embeddings_final[i].insert(0, texts[i])

with open('200 labelled embeddings only first 384 words.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(embeddings_final)

In [52]:
# Dataset from ben Horne
bendata = pd.read_csv('Transfer/article_matches.csv')
bendata.head(3)
print(bendata.shape)

(19386, 3)


In [53]:
bendata = bendata[~bendata['content'].isna()]
suicide_Articles_from_ben_data = bendata['content'].tolist()

wordcount = 0.0
for art in suicide_Articles_from_ben_data:
    wordcount += len(art.split())

print("Average word count: ", wordcount/len(suicide_Articles_from_ben_data))

Average word count:  1126.8050869318474


In [54]:
print(bendata.shape)

(19383, 3)


In [55]:
import pickle

#Load sentences & embeddings from disc
with open('ben_embeddings.pkl', "rb") as fIn:
    stored_data = pickle.load(fIn)
    stored_bendata_sentences = stored_data['sentences']
    stored_bendata_embeddings = stored_data['embeddings']

In [56]:
print(len(stored_bendata_sentences))
print(len(stored_bendata_embeddings))

19383
19383


In [57]:
cos_sim_bendata_financial = util.cos_sim(stored_bendata_embeddings, financial_problem_averaged_embedding)
cos_sim_bendata_legal = util.cos_sim(stored_bendata_embeddings, legal_problem_averaged_embedding)
cos_sim_bendata_school = util.cos_sim(stored_bendata_embeddings, school_problem_averaged_embedding)
cos_sim_bendata_health = util.cos_sim(stored_bendata_embeddings, health_problem_averaged_embedding)

In [58]:
print("Max similarity for financial problem: ", torch.max(cos_sim_bendata_financial))
print("Max similarity for legal problem: ", torch.max(cos_sim_bendata_legal))
print("Max similarity for school problem: ", torch.max(cos_sim_bendata_school))
print("Max similarity for health problem: ", torch.max(cos_sim_bendata_health))

Max similarity for financial problem:  tensor(0.7427)
Max similarity for legal problem:  tensor(0.7710)
Max similarity for school problem:  tensor(0.8391)
Max similarity for health problem:  tensor(0.8231)


In [28]:
# print(suicide_Articles_from_ben_data[torch.argmax(cos_sim_bendata_financial).item()])
# print("next")
# print(suicide_Articles_from_ben_data[torch.argmax(cos_sim_bendata_legal).item()])
# print("next")
# print(suicide_Articles_from_ben_data[torch.argmax(cos_sim_bendata_school).item()])
# print("next")
# print(suicide_Articles_from_ben_data[torch.argmax(cos_sim_bendata_health).item()])

In [29]:
# all_embeddings = financial_problem_embeddings + legal_problem_embeddings + school_problem_embeddings + health_problem_embeddings

# labels = []
# for i in range(len(financial_problem_embeddings)):
#     labels.append(FinancialLabel)
# for i in range(len(legal_problem_embeddings)):
#     labels.append(Legallabel)
# for i in range(len(school_problem_embeddings)):
#     labels.append(SchoolLabel)
# for i in range(len(health_problem_embeddings)):
#     labels.append(HealthLabel)

# print(np.array(all_embeddings).shape)
# print(np.array(labels).shape)

In [59]:
all_avg_embeddings = []
all_avg_embeddings.append(financial_problem_averaged_embedding)
all_avg_embeddings.append(legal_problem_averaged_embedding)
all_avg_embeddings.append(school_problem_averaged_embedding)
all_avg_embeddings.append(health_problem_averaged_embedding)
print(np.array(all_avg_embeddings).shape)

cos_sim_bendata_avg_embeddings = util.cos_sim(stored_bendata_embeddings, all_avg_embeddings)
print(np.array(cos_sim_bendata_avg_embeddings).shape)

(4, 768)
(19383, 4)


/Users/minijain/Library/Python/3.9/lib/python/site-packages/sentence_transformers/util.py:39: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:264.)
  b = torch.tensor(b)


In [31]:
# #MAJORITY VOTING BASED CLASSIFICATION

# #Find the pairs with the highest cosine similarity scores
# # IMPORTANT - COSINE SIMILARITY FUNCTION SHOULD BE BASSED CORRECT EMBEDDINGS ARRAY
# cosine_scores = util.cos_sim(stored_bendata_embeddings, all_embeddings)
# print(len(cosine_scores))
# pairs = []
# for i in range(len(cosine_scores)):
#     for j in range(len(cosine_scores[0])):
#         pairs.append({'index': [i, j], 'score': cosine_scores[i][j]})

# #Sort scores in decreasing order
# pairs = sorted(pairs, key=lambda x: x['score'], reverse=True)

# print(np.array(pairs).shape)

In [32]:
# from collections import Counter
# topk = 5
# threshhold = 0.5
# with open('ben_dataset_all_class_scores_individual_embeddings.csv', 'w', newline='') as file:
#     writer = csv.writer(file)
#     writer.writerow(["Id", "Article", FinancialLabel, Legallabel, SchoolLabel, HealthLabel])
#     for i in range(len(cosine_scores)):
#         articlestats = cosine_scores[i]
#         financial_score = torch.max(articlestats[np.argwhere(np.array(labels) == FinancialLabel).flatten()]).item()
#         legal_score = torch.max(articlestats[np.argwhere(np.array(labels) == Legallabel).flatten()]).item()
#         school_score = torch.max(articlestats[np.argwhere(np.array(labels) == SchoolLabel).flatten()]).item()
#         health_score = torch.max(articlestats[np.argwhere(np.array(labels) == HealthLabel).flatten()]).item()        
#         arr = [i+1, stored_bendata_sentences[i],financial_score, legal_score, school_score, health_score]
#         writer.writerow(arr)


In [60]:
#AVERAGING BASED CLASSIFICATION

#Find the pairs with the highest cosine similarity scores
# IMPORTANT - COSINE SIMILARITY FUNCTION SHOULD BE BASSED CORRECT EMBEDDINGS ARRAY
cosine_scores = util.cos_sim(stored_bendata_embeddings, all_avg_embeddings)
print(np.array(cosine_scores).shape)
pairs = []
for i in range(len(cosine_scores)):
    for j in range(len(cosine_scores[0])):
        pairs.append({'index': [i, j], 'score': cosine_scores[i][j]})

#Sort scores in decreasing order
pairs = sorted(pairs, key=lambda x: x['score'], reverse=True)

print(np.array(pairs).shape)

(19383, 4)
(77532,)


In [61]:
texts = [FinancialLabel, Legallabel, SchoolLabel, HealthLabel] 

In [35]:
# headerrow = ["Score","Label", "Article"]
# rows = []
# for pair in pairs[0:20]:
#     i, j = pair['index']
#     rows.append([pair['score'].item(), texts[j], stored_bendata_sentences[i]])

# with open('ben_dataset_analysis_top20_highest_matches_avg_embeddings.csv', 'w', newline='') as file:
#     writer = csv.writer(file)
#     writer.writerow(headerrow)
#     writer.writerows(rows)


In [62]:
cosine_scores = util.cos_sim(stored_bendata_embeddings, all_avg_embeddings)
with open('ben_dataset_all_class_scores_embeddings_using_first_384_words.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Id", "Article", FinancialLabel, Legallabel, SchoolLabel, HealthLabel])
    for i in range(len(cosine_scores)):
        articlestats = cosine_scores[i]
        arr = [i+1, stored_bendata_sentences[i],articlestats[0].item(), articlestats[1].item(), articlestats[2].item(), articlestats[3].item()]
        writer.writerow(arr)